### Make data table (new row for every scan / treatment / surgery event), convert days to dates, add new columns, and impute missing values

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import os

# import + load data 
data_file_path = r"C:\Users\mm17b2k.DS\Documents\ARCANE_Data\Data\LTH22069_cohort_details_jitteredAge.txt"
surgery_file_path = r"C:\Users\mm17b2k.DS\Documents\ARCANE_Data\Data\LTH22069_DR2_surgeryDetails_2.txt"

pd.set_option('display.max_columns', None)
data = pd.read_csv(data_file_path, sep="|")
data.rename(columns={'pseudoasscessionid': 'pseudoacccessionid'}, inplace=True)


# convert diagnosis date from quarter to month, in format Q/YYYY to MM/YYYY
dates = data['DiagnosisDate_QQYYYY'].to_list()
new_dates = []
for date in dates:
    new_dates.append(str((int(date[0])-1)*3 + 1) + date[1:])

# add estimated dates into dataframe 
data['DiagnosisDate_MMYYYY'] = new_dates
data['DiagnosisDate_MMYYYY'] = pd.to_datetime(data['DiagnosisDate_MMYYYY'])

data['ScanDate'] = data['DiagnosisDate_MMYYYY'] + pd.to_timedelta(data['ScanDaysFromDiagnosis'], unit='D')

data['ChemoDate'] = data['DiagnosisDate_MMYYYY'] + pd.to_timedelta(data['starttherapydaysfromdiagnosis'], unit='D')

data['RadioDate'] = data['DiagnosisDate_MMYYYY'] + pd.to_timedelta(data['starttherapydaysfromdiagnosis2'], unit='D')


# add date of primary diagnosis, when disease phase is definitive/primary diagnosis / or earliest date if no def/pri diag
patient_ids = data['shortpatpseudoid'].unique().tolist() # extract patient ids as list
patient_id_without_primary_diag = []
primary_diag_dates_dict = {}
for pat in patient_ids:
    patient_data = data[data['shortpatpseudoid'] == pat].reset_index()
    def_primary_diag = False
    for i, row in enumerate(patient_data['DiagnosisDate_MMYYYY']):
        def_primary_diag = False
        if patient_data['DiseasePhaseLabel'][i] == 'Definitive/Primary Diagnosis':
            def_primary_diag = True
            primary_diag_dates_dict[pat] = row
            break
    if not def_primary_diag:
        if pat not in patient_id_without_primary_diag:
            patient_id_without_primary_diag.append(pat)
        primary_diag_dates_dict[pat] = patient_data['DiagnosisDate_MMYYYY'].min()

# examples with multiple diagnosis dates - current assumption is surgery is days after the later primary diagnosis date
pat_multiple_diag = []
for pat in patient_ids:
    pat_data = data[data['shortpatpseudoid'] == pat]
    diag_date = []
    for row in pat_data.iterrows():
        if row[1]['DiseasePhaseLabel'] == 'Definitive/Primary Diagnosis':
            diag_date.append(row[1]['DiagnosisDate_MMYYYY'])
    diag_date = sorted(list(set(diag_date)))
    if len(diag_date) > 1:
        primary_diag_dates_dict[pat] = diag_date[-1] # pick latest date
        pat_multiple_diag.append(pat)
        #print('This patient had two primary diagnosis dates', pat, diag_date, diag_date[-1])

data['PrimaryDiagnosisDate'] = data['shortpatpseudoid'].map(primary_diag_dates_dict)


# add death date as time after primary diagnosis
data['DeathDate'] = pd.to_datetime(data['PrimaryDiagnosisDate']) + pd.to_timedelta(data['deathdaysfromprimarydiagnosis'], unit='D')


def impute_if_missing(col):
    global data 
    n = 0
    for pat in patient_ids:
        patient_data = data[data['shortpatpseudoid'] == pat]
        total = len(patient_data[col])
        na_count = patient_data[col].isna().sum()
        unique_vals = patient_data[col].dropna().values
        
        if len(unique_vals) > 0: # only change if there is at least one real value (leave as na if all na)
            # choose earlier death date as we are measuring from later diagnosis date (as the longer days to death was meant to be applied to the earlier diagnosis date)
            if col in ['DeathDate', 'deathdaysfromprimarydiagnosis', 'PrimaryDiagnosisDate']:
                changed_count = (unique_vals != min(unique_vals)).sum()
                # if changed_count > 0:
                #     print(pat, unique_vals)
                data.loc[data['shortpatpseudoid'] == pat, col] = min(unique_vals)
            elif col in ['jitteredageatprimarydiagnosis']:
                changed_count = (unique_vals != max(unique_vals)).sum()
                data.loc[data['shortpatpseudoid'] == pat, col] = max(unique_vals)
            else:
                changed_count = (unique_vals != unique_vals[0]).sum()
                data.loc[data['shortpatpseudoid'] == pat, col] = unique_vals[0]
            n += (changed_count + na_count)
    #print(f"Changed {changed_count} non-NA values") #-> all 0
    print(f"Imputed missing values for {n} patients in {col}")

impute_if_missing('jitteredageatprimarydiagnosis')
impute_if_missing('DeathDate')
impute_if_missing('deathdaysfromprimarydiagnosis')
# none missing for the below
# impute_if_missing('sex')
# impute_if_missing('broadethnicgroup')
# impute_if_missing('PrimaryDiagnosisDate')

# Result: 
# Imputed missing values for 1712 patients in jitteredageatprimarydiagnosis
# Imputed missing values for 1479 patients in DeathDate
# Imputed missing values for 1479 patients in deathdaysfromprimarydiagnosis


# Add surgery days and dates
surgery_data = pd.read_csv(surgery_file_path, sep=",")

# convert diagnosis date from quarter to month, in format QQ/YYYY to MM/YYYY
dates = surgery_data['DiagnosisDate_QQYYYY'].to_list()
new_dates = []
for date in dates:
    new_dates.append(str((int(date[1])-1)*3 + 1) + date[2:])
# add estimated dates into dataframe 
surgery_data['DiagnosisDate_MMYYYY'] = new_dates
surgery_data['SurgeryDates'] = pd.to_datetime(surgery_data['DiagnosisDate_MMYYYY']) + pd.to_timedelta(surgery_data['surgeryDaysFromDiagnosis'], unit='D')

surgery_data = surgery_data.groupby('patpseudoid')[['DiagnosisDate_MMYYYY', 'surgeryDaysFromDiagnosis', 'SurgeryDates']].agg(list).reset_index()


surgery_days_dict = {}
surgery_dates_dict = {}
for pat in surgery_data['patpseudoid']:
    surgery_days_dict[pat] = sorted(surgery_data[surgery_data['patpseudoid'] == pat]['surgeryDaysFromDiagnosis'].values[0])
    surgery_dates_dict[pat] = sorted(surgery_data[surgery_data['patpseudoid'] == pat]['SurgeryDates'].values[0])
   
data['SurgeryDaysFromDiagnosis'] = data['patpseudoid'].map(surgery_days_dict)
data['SurgeryDates'] = data['patpseudoid'].map(surgery_dates_dict)

    
# Data with two lots of pathology - removed but maybe put back in later?
# N and M stage increase and chemo added so remove earlier entries: [2652, 2653, 2654, 2655]
#print(data[data['shortpatpseudoid'] == 'X'][['shortpatpseudoid', 'shortpseudoaccessionid', 'ScanDate', 'ChemoDate', 'RadioDate']].sort_values(by=['ScanDate', 'shortpseudoaccessionid']))
# M stage changes NaN to M1a so remove [5739, 5740, 5741]
#print(data[data['shortpatpseudoid'] == 'Y'][['shortpatpseudoid', 'shortpseudoaccessionid', 'ScanDate', 'ChemoDate', 'RadioDate']].sort_values(by=['ScanDate', 'shortpseudoaccessionid']))
data = data.drop([2652, 2653, 2654, 2655, 5739, 5740, 5741]).reset_index(drop=True)

In [3]:
# number of unique patient ids
print('Number of unique patient ids:', len(data['shortpatpseudoid'].unique()))

Number of unique patient ids: 1284


In [ ]:
# find all with tumour label 0
T0_ids = []
for row in data.iterrows():
    if row[1]['TumourLabel'] == '0':
        if row[1]['shortpatpseudoid'] not in T0_ids:
            T0_ids.append(row[1]['shortpatpseudoid'])
print(T0_ids)
print('Number of patients with TumourLabel 0:', len(T0_ids)) # 12

### Make grouped data table that has a new row for every scan with the treatment, surgery and pathology aggregated across all scans for each patient

In [ ]:
# rows are aggregated if they have the same values for the patient id / scan date / diagnosis date / patient demographics and tumour characteristics  (as all of these have no missing data and are the same for each patient) 

# columns are aggregated by taking the first non-null value across the rows for each patient
# for the treatment dates, add all the unique dates for each patient as a set

# columns with no missing data
columns = []
for col in data.columns:
    missing_values = data[col].isnull().sum()
    #print("Number of missing values in '{}':".format(col), missing_values)
    if missing_values == 0:
        # don't include these as they change for each new treatment/scan
        if col not in ['Unnamed: 0', 'pseudoaccessionid', 'shortpseudoaccessionid']:
            columns.append(col)
print('The following columns, with no missing data are used to aggregate data:', columns)

def first_non_null(lst):
    for item in lst:
        if type(item) == list:
            return item 
        if not pd.isna(item):
            return item
    return np.nan

# when aggregating the  rows, keep the first non-null value
# as it can have NaN for some and then the same value for the rest across the rows for each patient 
first_columns = ['shortpseudoaccessionid', 'deathdaysfromprimarydiagnosis', 'broadethnicgroup', 'DiseaseProgressionLabel', 'GradeCodeLabel', 'Stage', 'StageLabel', 'ConfirmationLabel', 'TumourLabel', 'NodeLabel', 'MetastasisLabel', 'StageSystemLabel', 'starttherapylabel1', 'starttherapylabel2', 'jitteredageatprimarydiagnosis', 'DeathDate', 'SurgeryDaysFromDiagnosis', 'SurgeryDates']

# keep all the treatment dates and add as a set for each new aggregated row 
set_columns =['starttherapydaysfromdiagnosis', 'starttherapydaysfromdiagnosis2', 'ChemoDate', 'RadioDate']

# check the following columns that the first non-null values are then the same across all the accession events corresponding to a given scan.
# i.e. the aggregation is not losing any data for these columns 
test = False
if test == True:
    cols_to_check = ['shortpseudoaccessionid','deathdaysfromprimarydiagnosis', 'broadethnicgroup', 'DiseaseProgressionLabel', 'GradeCodeLabel', 'Stage', 'StageLabel', 'ConfirmationLabel', 'TumourLabel', 'NodeLabel', 'MetastasisLabel', 'StageSystemLabel', 'starttherapylabel1', 'starttherapylabel2', 'jitteredageatprimarydiagnosis', 'DeathDate']
    # these columns had multiple values: 'GradeCodeLabel, 'NodeLabel', 'MetastasisLabel' 
    # - removed the specific examples as they were incorrect data
else:
    cols_to_check = []
    
for item in cols_to_check:
    first_columns.remove(item)
    set_columns.append(item)
    
aggregation = {col: first_non_null for col in first_columns}
aggregation.update({col: set for col in set_columns})
# group by the columns with no missing data and aggregate the rows using strategy defined above 
grouped_data = data.groupby(columns).agg(aggregation).reset_index()

# check that the number of accession events is the same
assert grouped_data['shortpseudoaccessionid'].unique().shape == data['shortpseudoaccessionid'].unique().shape

# check that the columns with multiple values are the same across all the accession events corresponding to a given scan
if test == True:
    for i, row in grouped_data.iterrows():
        for col in cols_to_check:
            if len(set(row[col])) > 1:
                for item in row[col]:
                    if not pd.isna(item):
                        print(col, row[col], row['shortpatpseudoid'])
    print('These columns have the same value for all rows corresponding to a given scan')

# impute/combine chemo and radio days/dates
chemo_dates_dict = {}
radio_dates_dict = {}
chemo_days_dict = {}
radio_days_dict = {}
for pat in patient_ids:
    pat_data = grouped_data[grouped_data['shortpatpseudoid'] == pat]
    chemo_dates = pat_data['ChemoDate'].tolist()
    radio_dates = pat_data['RadioDate'].tolist()
    chemo_days = pat_data['starttherapydaysfromdiagnosis'].tolist()
    radio_days = pat_data['starttherapydaysfromdiagnosis2'].tolist()

    def add_unique_dates(dates):
        # add all the unique dates for each patient as a set
        unique_dates = []
        for date in dates:
            if list(date) != [pd.NaT]:
                for d in list(date):
                    if d == d:
                        if d not in unique_dates:
                            unique_dates.append(d)
        return unique_dates

    unique_chemo_dates = add_unique_dates(chemo_dates)
    unique_radio_dates = add_unique_dates(radio_dates)
    unique_chemo_days = add_unique_dates(chemo_days)
    unique_radio_days = add_unique_dates(radio_days)

    if unique_chemo_dates != []:
        chemo_dates_dict[pat] = sorted(unique_chemo_dates)
    if unique_radio_dates != []:
        radio_dates_dict[pat] = sorted(unique_radio_dates)
    if unique_chemo_days != []:
        chemo_days_dict[pat] = sorted(unique_chemo_days)
    if unique_radio_days != []:
        radio_days_dict[pat] = sorted(unique_radio_days)

grouped_data['ChemoDate'] = grouped_data['shortpatpseudoid'].map(chemo_dates_dict)
grouped_data['RadioDate'] = grouped_data['shortpatpseudoid'].map(radio_dates_dict)
grouped_data['starttherapydaysfromdiagnosis'] = grouped_data['shortpatpseudoid'].map(chemo_days_dict)
grouped_data['starttherapydaysfromdiagnosis2'] = grouped_data['shortpatpseudoid'].map(radio_days_dict)


# add column for cancer surgery dates that are from 3 weeks before primary diagnosis 
# accounting for the cases where primary diagnosis is from pathology after surgery
# note this will be certain when we have surgery type
patients = []
i, j, k = 0, 0, 0
cancer_surgery_dates_dict = {}
non_cancer_surgery_dates_dict = {}
for pat in patient_ids:
    # exclude patients which have primary diagnosis imputed as the first diagnosis date given (recurrence/metastasis)
    if pat in patient_id_without_primary_diag:
        continue
    pat_data = grouped_data[grouped_data['shortpatpseudoid'] == pat]
    non_cancer_surgery_dates = []
    cancer_surgery_dates = []
    row = pat_data.iloc[0]
    first_diag_date = min(pat_data['DiagnosisDate_MMYYYY'])
    # if the patient has had any surgery
    if row['SurgeryDates'] == row['SurgeryDates']: #pd.isna
        for surgery_date in row['SurgeryDates']:
            if not surgery_date > first_diag_date - pd.Timedelta(days=3*7):
                non_cancer_surgery_dates.append(surgery_date)
                if row['shortpatpseudoid'] not in patients:
                    patients.append(row['shortpatpseudoid'])
            else:
                cancer_surgery_dates.append(surgery_date)

    # check all surgery dates are accounted for
    if row['SurgeryDates'] == row['SurgeryDates']:
        assert len(non_cancer_surgery_dates) + len(cancer_surgery_dates) == len(row['SurgeryDates'])

    # add entries to dictionary and count how many patients in each group
    if non_cancer_surgery_dates != []:
        i+=1
        non_cancer_surgery_dates_dict[pat] = sorted(non_cancer_surgery_dates)
        #print('diagnosis', row['PrimaryDiagnosisDate'], 'non_cancer', non_cancer_surgery_dates, 'cancer', cancer_surgery_dates, row['shortpatpseudoid'])
        
    if cancer_surgery_dates != []:
        j+=1
        cancer_surgery_dates_dict[pat] = sorted(cancer_surgery_dates)

    if non_cancer_surgery_dates == [] and cancer_surgery_dates == []:
        k+=1
        #print('non_cancer', non_cancer_surgery_dates, 'cancer', cancer_surgery_dates, 'all', row['PrimaryDiagnosisDate'], row['shortpatpseudoid'])

print('Number of patients with non-cancer surgery dates:', i)
print('Number of patients with cancer surgery dates:', j)
print('Number of patients with no surgery dates:', k+len(patient_id_without_primary_diag))

grouped_data['CancerSurgeryDates'] = grouped_data['shortpatpseudoid'].map(cancer_surgery_dates_dict)
grouped_data['CancerSurgeryDates'] = grouped_data['CancerSurgeryDates'].apply(lambda x: np.nan if x != x else x)



# find patients which did not have and radio or chemo before cancer surgery
no_chemo_before_surgery = []
no_radio_before_surgery = []
no_pre_treatment = []
no_treatment = []
for pat in patient_ids:
    pat_data = grouped_data[grouped_data['shortpatpseudoid'] == pat]
    first_row = pat_data.iloc[0]
    cancer_surgery_dates = first_row['CancerSurgeryDates']
    chemo_dates = first_row['ChemoDate']
    radio_dates = first_row['RadioDate']

    # if there was no chemo or radiotherapy at all
    if cancer_surgery_dates == cancer_surgery_dates:
        if chemo_dates != chemo_dates:
            if pat not in no_chemo_before_surgery:
                no_chemo_before_surgery.append(pat)
        if radio_dates != radio_dates:
            if pat not in no_radio_before_surgery:
                no_radio_before_surgery.append(pat)
        if pat in no_chemo_before_surgery and pat in no_radio_before_surgery:
            no_treatment.append(pat)

    # if there was no chemo or radiotherapy before surgery but was after      
    if cancer_surgery_dates == cancer_surgery_dates:
        if chemo_dates == chemo_dates:
            if chemo_dates[0] >= cancer_surgery_dates[0]:
                if pat not in no_chemo_before_surgery:
                    no_chemo_before_surgery.append(pat)

    if cancer_surgery_dates == cancer_surgery_dates:
        if radio_dates == radio_dates:
            if radio_dates[0] >= cancer_surgery_dates[0]:
                if pat not in no_chemo_before_surgery:
                    no_radio_before_surgery.append(pat)

for pat in patient_ids:
    if pat in no_chemo_before_surgery:
        if pat in no_radio_before_surgery:
            no_pre_treatment.append(pat)

treatment_after_surgery = []
for p in no_pre_treatment:
    if p not in no_treatment:
        treatment_after_surgery.append(p)

print('no chemo before surgery', len(no_chemo_before_surgery))
print('no radio before surgery', len(no_radio_before_surgery))
print('no radio or chemo at all', len(no_treatment))
print('no radio or chemo until after surgery', len(treatment_after_surgery))
print('no pre-treatment', len(no_pre_treatment))
# no chemo before surgery 805
# no radio before surgery 454
# no radio or chemo at all 343
# no radio or chemo until after surgery 86
# no pre-treatment 429

# add NoPreTreatment column to grouped_data
grouped_data['NoPreTreatment'] = grouped_data['shortpatpseudoid'].apply(lambda x: True if x in no_pre_treatment else False)


# compile pathology data incase there is multiple
pathology_data = {}
pats = []
for pat in patient_ids:
    pathology_data_patient = {}
    pat_data = grouped_data[grouped_data['shortpatpseudoid'] == pat]
    for row in pat_data.iterrows():
        if row[1]['ConfirmationLabel'] == 'Histology - primary':
            if row[1]['DiagnosisDate_MMYYYY'] not in pathology_data_patient:
                pathology_data_patient[row[1]['DiagnosisDate_MMYYYY']] = [row[1]['TumourLabel'], row[1]['NodeLabel'], row[1]['MetastasisLabel'], row[1]['GradeCodeLabel'], row[1]['MorphologyLabel'], row[1]['Stage']]
                
    if len(pathology_data_patient) > 1:
        pats.append(pat)
    pathology_data[pat] = pathology_data_patient

grouped_data['PathologyData'] = grouped_data['shortpatpseudoid'].map(pathology_data)


grouped_data#.describe()

The following columns, with no missing data are used to aggregate data: ['patpseudoid', 'sex', 'DiagnosisDate_QQYYYY', 'DiseasePhaseLabel', 'CareEpisodeID', 'SiteLabel', 'siteCDS', 'MorphologyLabel', 'ICD10Label', 'Examination', 'pseudoacccessionid', 'ScanDaysFromDiagnosis', 'shortpatpseudoid', 'DiagnosisDate_MMYYYY', 'ScanDate', 'PrimaryDiagnosisDate']

Number of patients with non-cancer surgery dates: 214
Number of patients with cancer surgery dates: 989
Number of patients with no surgery dates: 261
no chemo before surgery 793
no radio before surgery 453
no radio or chemo at all 343
no radio or chemo until after surgery 85
no pre-treatment 428

### Filter the data to only include patients with an MRI scan within 16 weeks before surgery, N stage pathology and without no neo-adjuvant treatment before surgery. Also remove multiple scans from same patient to have only the last one before surgery.

In [ ]:
cohort_1 = grouped_data[(grouped_data['ConfirmationLabel']=='Histology - primary')
             &(grouped_data['NodeLabel']==grouped_data['NodeLabel'])
             &~(grouped_data['NodeLabel'].isin(['X']))
             &~(grouped_data['TumourLabel'].isin(['0', 'X']))
             &(grouped_data['CancerSurgeryDates']==grouped_data['CancerSurgeryDates'])
             &(grouped_data['NoPreTreatment']==True)
             &( (grouped_data['Examination']=='MRECT') | (grouped_data['Examination']=='MPELV') )
               ][['shortpatpseudoid', 'ScanDate', 'CancerSurgeryDates', 'TumourLabel', 'NodeLabel', 'MetastasisLabel', 'Stage', 'GradeCodeLabel', 'MorphologyLabel', 'ChemoDate', 'RadioDate', 'DiagnosisDate_MMYYYY', 'PrimaryDiagnosisDate', 'ConfirmationLabel', 'Examination', 'sex', 'jitteredageatprimarydiagnosis', 'DeathDate', 'broadethnicgroup', 'shortpseudoaccessionid', 'SurgeryDates', 'PathologyData', 'patpseudoid']].sort_values(by=['shortpatpseudoid', 'ScanDate'])


cohort_1_pat_ids = cohort_1['shortpatpseudoid'].unique()
rows_removed = 0
old_scans_num = 0
for pat in cohort_1_pat_ids:
    pat_data = cohort_1[cohort_1['shortpatpseudoid'] == pat]
    for row in pat_data.iterrows():
        # remove all the scans after the last cancer surgery date
        if row[1]['ScanDate'] > row[1][('CancerSurgeryDates')][-1]:
            #print(pat, row[1]['ScanDate'], row[1]['CancerSurgeryDates'])
            cohort_1.drop(row[0], inplace=True) # remove scan / row from cohort_1
            rows_removed += 1
        
        # remove any scans that are older than 16 weeks before the first surgery
        if row[1]['ScanDate'] < row[1]['CancerSurgeryDates'][0] - pd.Timedelta(days=16*7):
            cohort_1.drop(row[0], inplace=True)
            old_scans_num += 1

        # if row[1]['shortpatpseudoid'] in [9EFBDBE0E5']: 
        #     cohort_1.drop(row[0], inplace=True)
print('Number of scans more than 16 weeks before first surgery:', old_scans_num)
print('Patients with no pre-surgery scan:', len(cohort_1_pat_ids) - len(cohort_1['shortpatpseudoid'].unique()))
print('Scans removed as after last surgery:', rows_removed)


# Surgery timeline logic to remove unwanted scans
removed = 0
i, j, k = 0, 0, 0
post_first_surgery_scans = []
for row in cohort_1.iterrows():
    if len(row[1]['CancerSurgeryDates']) > 1:
        if row[1]['ScanDate'] > row[1][('CancerSurgeryDates')][0]:
            post_first_surgery_scans.append(row[1]['shortpatpseudoid'])
            if row[1]['ScanDate'] < row[1][('CancerSurgeryDates')][-1]:
                # check patients with more than 4 surgery dates
                if len(row[1]['CancerSurgeryDates']) >= 4:
                    #cohort_1.drop(row[0], inplace=True)
                    print('check patient {}'.format(row[1]['shortpatpseudoid']))
                    
                # remove scans that are more than 16 weeks before the 2nd surgery
                if row[1][('CancerSurgeryDates')][0] < row[1]['ScanDate'] < row[1][('CancerSurgeryDates')][1]:
                    if row[1][('CancerSurgeryDates')][1] - row[1]['ScanDate'] > pd.Timedelta(days=16*7):
                        cohort_1.drop(row[0], inplace=True)
                        removed += 1
                        j += 1
                        #print('removed as scan date: {} is more than 16 weeks before 2nd surgery {}'.format(row[1]['ScanDate'], row[1]['CancerSurgeryDates'][1]))
                        
                # remove scans that are more than 16 weeks before the 3rd surgery
                if len(row[1]['CancerSurgeryDates']) > 2:
                    if row[1][('CancerSurgeryDates')][1] < row[1]['ScanDate'] < row[1][('CancerSurgeryDates')][2]:
                        if row[1][('CancerSurgeryDates')][2] - row[1]['ScanDate'] > pd.Timedelta(days=16*7):
                            cohort_1.drop(row[0], inplace=True)
                            removed += 1
                            k += 1
        
    
print('Number of scans removed becuase cant tell surgery timeline:', removed)
print('Number of scans removed as post surgery 1 scan is more than 16 weeks before 2nd surgery:', j)
print('Number of scans removed as post surgery 2 scan is more than 16 weeks before 3rd surgery:', k)

# remove any scans that are not the last scan before surgery
cohort_1_pat_ids = cohort_1['shortpatpseudoid'].unique()
pats = []
for pat in cohort_1_pat_ids:
    pat_data = cohort_1[cohort_1['shortpatpseudoid'] == pat]
    # if more than one row
    if len(pat_data) > 1:
        pats.append(pat)
# ['24D14FE845', '35BC78A229', '43E6984256', 'C56C583A82', 'DA4E559B82', 'F848962798']

keep_first_scan = ['43E6984256', 'DA4E559B82', 'F848962798', '9FFF37F519']
keep_second_scan = ['24D14FE845', 'C56C583A82']
keep_none = ['35BC78A229']
# clear which scans to keep: 24D14FE845, 43E6984256, C56C583A82
#  '35BC78A229' is post chemo


num_scans_removed = 0
for i, pat in enumerate(pats):
    pat_data = cohort_1[cohort_1['shortpatpseudoid'] == pat].sort_values(by=['ScanDate'])
    #print('surgery date/s:', pat_data['CancerSurgeryDates'].tolist()[0])
    if pat in keep_first_scan:
        idx = 0
        for row in pat_data.iterrows():
            #if idx == 0:
            #    print('keeping scan date:', row[1]['ScanDate'])
            if idx != 0:
                #print('removing scan date:', row[1]['ScanDate'])
                cohort_1.drop(row[0], inplace=True)
                num_scans_removed += 1
            idx += 1
    if pat in keep_second_scan:
        idx = 0
        for row in pat_data.iterrows():
            #if idx == 1:
            #    print('keeping scan date:', row[1]['ScanDate'])
            if idx != 1:
                #print('removing scan date:', row[1]['ScanDate'])
                cohort_1.drop(row[0], inplace=True)
                num_scans_removed += 1
            idx += 1
    if pat in keep_none:
        print(pat)
        for row in pat_data.iterrows():
            #print('removing scan date:', row[1]['ScanDate'])
            cohort_1.drop(row[0], inplace=True)
            num_scans_removed += 1
print('Scans removed as not most recent:', num_scans_removed)


print('Number of patients in cohort 1:', len(cohort_1['shortpatpseudoid'].unique()))
print('Number of nodes', cohort_1['NodeLabel'].value_counts())
print('Tumour stages:', cohort_1['TumourLabel'].value_counts())
print('Number of patients with positive nodes:', 22+10+5+2+1+1)

cohort_1['sex_numeric'] = cohort_1['sex'].map({'Male': 0, 'Female': 1})
max = cohort_1['jitteredageatprimarydiagnosis'].max()
min = cohort_1['jitteredageatprimarydiagnosis'].min()
# min max scaling of jittered age
cohort_1['age_scaled'] = (cohort_1['jitteredageatprimarydiagnosis'] - min) / (max - min)

cohort_1['TumourLabel_numeric'] = cohort_1['TumourLabel'].map({'1': 0, '2': 0.33, '3': 0.67, '3a': 0.67, '4': 1, '4b': 1})


cohort_1.sort_values(by=['shortpatpseudoid', 'ScanDate'])

# save as excel
cohort_1.to_excel(r"C:\Users\mm17b2k.DS\Documents\ARCANE_Data\Cohort1.xlsx")

Number of scans more than 16 weeks before first surgery: 6
Patients with no pre-surgery scan: 10
Scans removed as after last surgery: 60

Number of scans removed because cant tell surgery timeline: 11
Number of scans removed as post surgery 1 scan is more than 16 weeks before 2nd surgery: 8
Number of scans removed as post surgery 2 scan is more than 16 weeks before 3rd surgery: 3

Scans removed as not most recent: 9
Number of patients in cohort 1: 195
Number of patients with positive nodes: 41

Number of nodes NodeLabel
0     154
1      22
2      10
1a      5
1b      2
2a      1
2b      1


Tumour stages: TumourLabel
2     94
3     52
1     36
4     11
4b     1
3a     1

In [ ]:
def get_important_data_by_patient_id(id):
    return data[data['shortpatpseudoid'] == id][['shortpatpseudoid', 'shortpseudoaccessionid', 'DiagnosisDate_MMYYYY', 'ScanDaysFromDiagnosis', 'ScanDate',  'ChemoDate', 'RadioDate', 'Examination', 'ConfirmationLabel', 'Stage', 'TumourLabel', 'NodeLabel', 'MetastasisLabel', 'DiseasePhaseLabel']].sort_values(by=['ScanDate', 'shortpseudoaccessionid'])

def get_full_data_patient_id(id):
    return data[data['shortpatpseudoid'] == id]


get_important_data_by_patient_id(id = "X")

In [ ]:
from tableone import TableOne
columns, categorical = [], []
for i,col in enumerate(grouped):
    print(i,col)
    if i not in [2,3,4,7,11,13,15,27,28,29,30]:    #[0, 1, 5, 9, 11, 12, 13, 14, 26, 28, 29]:
        columns.append(col)
        if grouped[col].dtype == 'object':
            categorical.append(col)
    
table = TableOne(grouped, columns=columns, categorical=categorical) #, groupby='patpseudoid', pval=True)
table.to_excel('tableone_grouped.xlsx')